## Xception model training

### import libs

In [1]:
import os
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from keras import optimizers
from keras import regularizers
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 21
nb_cpus = 8
nb_gpus = 4
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3'

image_size = (299, 299)
input_shape= (299, 299, 3)

train_path = "/home/nvme/liyu/xcp-batch6.4/CELLS-half/train"
valid_path = "/home/nvme/liyu/xcp-batch6.4/CELLS-half/valid"

### pretrain model

#### 1. load custom trained model

In [ ]:
custom_nb_classes = 20
input_shape= (299, 299, 3)

with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(custom_nb_classes, activation='softmax', name="predictions")(p_out)

    custom_model = Model(inputs=base_model.input, outputs=predictions)
    
    custom_model.load_weights('batch6.1_finetune_010.h5')

#### 2. load custom model weights to new model, layer by layer

In [ ]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    # m_out = SeparableConv2D(4096, kernel_size=3, strides=2)(m_out)
    # m_out = BatchNormalization()(m_out)
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)
    
    # load weights layer by layer
    for i in range(len(custom_model.layers)-1):
        model.layers[i].set_weights(custom_model.layers[i].get_weights())
#     model.load_weights('batch6.1_finetune_010.h5', by_name=True)

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
else:
    parallel_model = model
    
optimizer = optimizers.SGD(lr=0.005, momentum=0.9, decay=0.0003)
parallel_model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

In [ ]:
del custom_model

In [ ]:
# parallel_model.load_weights('weights_010_0.0698.hdf5')

In [ ]:
# model.save_weights('batch6.1_finetune_010.h5')

In [ ]:
batch_size = 128
epochs = 1

In [ ]:
img_gen_t = ImageDataGenerator(rotation_range=30,                            
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               brightness_range=[0.8, 1.2],
                               horizontal_flip=True,
                               vertical_flip=True)
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator), 
                             workers=nb_cpus, 
                             use_multiprocessing=True)

model.save_weights("Xception_first_train.h5")

### train model

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    # m_out = SeparableConv2D(4096, kernel_size=3, strides=1)(m_out)
    # m_out = BatchNormalization()(m_out)
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    model = Model(inputs=base_model.input, outputs=predictions)
    
#     model.load_weights("Xception_first_train.h5")

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
else:
    parallel_model = model
    
optimizer = optimizers.SGD(lr=0.002, momentum=0.9, decay=0.0003)
parallel_model.compile(optimizer=optimizer, 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [4]:
parallel_model.load_weights('batch6.3_009_0.0767.hdf5')

In [5]:
batch_size = 64
epochs = 100

In [6]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

img_gen_t = ImageDataGenerator(rotation_range=30,                            
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               brightness_range=[0.8, 1.2],
                               horizontal_flip=True,
                               vertical_flip=True)
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

checkpoint = ModelCheckpoint("batch6.4_{epoch:03d}_{val_loss:.4f}.hdf5", monitor='val_loss', verbose=1,
                             save_best_only=False, save_weights_only=True, mode='min', period=1)

tensorboard = TensorBoard(log_dir="./logs", histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=0.000001)

callbacks = [checkpoint, tensorboard, reduce_lr]


parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator), 
                             callbacks=callbacks, 
                             workers=nb_cpus, 
                             use_multiprocessing=True, 
#                              initial_epoch=3
                             )

Found 1570764 images belonging to 21 classes.
Found 43605 images belonging to 21 classes.
Epoch 1/100
24544/24544 [==============================] - 11759s 479ms/step - loss: 0.0599 - acc: 0.9795 - val_loss: 0.0660 - val_acc: 0.9771

Epoch 00001: saving model to batch6.4_001_0.0660.hdf5
Epoch 2/100
24544/24544 [==============================] - 11754s 479ms/step - loss: 0.0566 - acc: 0.9804 - val_loss: 0.0660 - val_acc: 0.9778

Epoch 00002: saving model to batch6.4_002_0.0660.hdf5
Epoch 3/100
24544/24544 [==============================] - 11766s 479ms/step - loss: 0.0560 - acc: 0.9806 - val_loss: 0.0667 - val_acc: 0.9772

Epoch 00003: saving model to batch6.4_003_0.0667.hdf5
Epoch 4/100
24544/24544 [==============================] - 11790s 480ms/step - loss: 0.0555 - acc: 0.9809 - val_loss: 0.0657 - val_acc: 0.9779

Epoch 00004: saving model to batch6.4_004_0.0657.hdf5
Epoch 5/100
24544/24544 [==============================] - 11801s 481ms/step - loss: 0.0554 - acc: 0.9809 - val_loss: 

Process ForkPoolWorker-159:
Process ForkPoolWorker-156:
Process ForkPoolWorker-153:
Process ForkPoolWorker-155:
Process ForkPoolWorker-160:
Process ForkPoolWorker-157:
Process ForkPoolWorker-158:
Process ForkPoolWorker-154:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sakulaki/anaconda3/envs/algo-work/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/sakulaki/anaconda3/envs/algo-work/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/sakulaki/anaconda3/envs/algo-work/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/sakulaki/anaconda3/envs/algo-work/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/sakulaki/anaconda3/envs/algo-wor

  File "/home/sakulaki/anaconda3/envs/algo-work/lib/python3.5/site-packages/scipy/ndimage/interpolation.py", line 458, in affine_transform
    output, order, mode, cval, None, None)
KeyboardInterrupt


KeyboardInterrupt: 

In [7]:
model.save_weights("batch6.4_train2_finetuned.h5")